## 보스턴 주택 가격 예측: 딥러닝을 이용한 회귀 분석

이 노트북은 Keras를 사용하여 보스턴 주택 가격 데이터셋으로 회귀 모델을 구축, 훈련 및 평가하는 과정을 다룹니다. 딥러닝이 분류 문제뿐만 아니라 수치 예측(회귀) 문제에도 어떻게 적용될 수 있는지 보여줍니다.

### 1. 라이브러리 임포트 및 데이터 로드
필요한 라이브러리를 임포트하고 Keras에 내장된 보스턴 주택 가격 데이터셋을 로드합니다. 데이터의 형태와 처음 몇 개의 샘플을 출력하여 구조를 확인합니다.

In [2]:
import tensorflow as tf
from keras.datasets import boston_housing

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

print("--- 데이터 형태 ---")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("\n--- 데이터 샘플 ---")
print("X_train first 5 samples:\n", X_train[:5])
print("\ny_train first 5 samples:\n", y_train[:5])

--- 데이터 형태 ---
X_train shape: (404, 13)
y_train shape: (404,)

--- 데이터 샘플 ---
X_train first 5 samples:
 [[1.23247e+00 0.00000e+00 8.14000e+00 0.00000e+00 5.38000e-01 6.14200e+00
  9.17000e+01 3.97690e+00 4.00000e+00 3.07000e+02 2.10000e+01 3.96900e+02
  1.87200e+01]
 [2.17700e-02 8.25000e+01 2.03000e+00 0.00000e+00 4.15000e-01 7.61000e+00
  1.57000e+01 6.27000e+00 2.00000e+00 3.48000e+02 1.47000e+01 3.95380e+02
  3.11000e+00]
 [4.89822e+00 0.00000e+00 1.81000e+01 0.00000e+00 6.31000e-01 4.97000e+00
  1.00000e+02 1.33250e+00 2.40000e+01 6.66000e+02 2.02000e+01 3.75520e+02
  3.26000e+00]
 [3.96100e-02 0.00000e+00 5.19000e+00 0.00000e+00 5.15000e-01 6.03700e+00
  3.45000e+01 5.98530e+00 5.00000e+00 2.24000e+02 2.02000e+01 3.96900e+02
  8.01000e+00]
 [3.69311e+00 0.00000e+00 1.81000e+01 0.00000e+00 7.13000e-01 6.37600e+00
  8.84000e+01 2.56710e+00 2.40000e+01 6.66000e+02 2.02000e+01 3.91430e+02
  1.46500e+01]]

y_train first 5 samples:
 [15.2 42.3 50.  21.1 17.7]


### 2. 데이터 전처리: 특성 스케일링
입력 특성(X)들은 서로 다른 범위의 값을 가집니다. 모델이 안정적으로 학습하기 위해 모든 특성들이 비슷한 범위를 갖도록 스케일링을 진행합니다. 여기서는 `Normalizer`를 사용하여 각 샘플(행)의 유클리드 노름(L2 norm)이 1이 되도록 조정합니다.

In [3]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
X_train_scaled = normalizer.fit_transform(X_train)
X_test_scaled = normalizer.transform(X_test) # 테스트 데이터는 훈련 데이터 기준으로 변환

print("--- 스케일링 후 데이터 샘플 ---")
print("X_train_scaled first 5 samples:\n", X_train_scaled[:5])

--- 스케일링 후 데이터 샘플 ---
X_train_scaled first 5 samples:
 [[2.41189924e-03 0.00000000e+00 1.59296858e-02 0.00000000e+00
  1.05284655e-03 1.20196720e-02 1.79453585e-01 7.78264954e-03
  7.82785541e-03 6.00787902e-01 4.10962409e-02 7.76718953e-01
  3.66343633e-02]
 [4.07923050e-05 1.54587284e-01 3.80378407e-03 0.00000000e+00
  7.77620881e-04 1.42595058e-02 2.94184285e-02 1.17486336e-02
  3.74757051e-03 6.52077269e-01 2.75446433e-02 7.40857215e-01
  5.82747215e-03]
 [6.34505528e-03 0.00000000e+00 2.34463745e-02 0.00000000e+00
  8.17384658e-04 6.43803764e-03 1.29537981e-01 1.72609359e-03
  3.10891154e-02 8.62722952e-01 2.61666721e-02 4.86441025e-01
  4.22293817e-03]
 [8.65407330e-05 0.00000000e+00 1.13392175e-02 0.00000000e+00
  1.12518247e-03 1.31897603e-02 7.53763011e-02 1.30768051e-02
  1.09241016e-02 4.89399752e-01 4.41333705e-02 8.67155186e-01
  1.75004108e-02]
 [4.74343543e-03 0.00000000e+00 2.32476643e-02 0.00000000e+00
  9.15778156e-04 8.18934295e-03 1.13541078e-01 3.29718668e-03
  3.0

### 3. 딥러닝 모델 정의
회귀 문제를 풀기 위한 `Sequential` 모델을 정의합니다.

- **입력층**: 13개의 특성을 받습니다 (`input_shape=(13,)`).
- **은닉층**: `relu` 활성화 함수를 사용하는 여러 개의 `Dense` 층으로 구성하여 복잡한 비선형 관계를 학습합니다.
- **출력층**: 단일 주택 가격을 예측해야 하므로, 활성화 함수가 없는 하나의 `Dense` 층으로 구성합니다. 이를 통해 어떤 값이든 자유롭게 예측할 수 있습니다.

모델 컴파일 시:
- **Optimizer**: `rmsprop`을 사용합니다.
- **Loss**: 회귀 문제의 대표적인 손실 함수인 `mse` (Mean Squared Error, 평균 제곱 오차)를 사용합니다.
- **Metrics**: `mae` (Mean Absolute Error, 평균 절대 오차)를 추가하여 훈련 과정을 모니터링합니다. MAE는 예측 오차를 직관적으로 이해하는 데 도움이 됩니다.

In [4]:
from keras import models, layers

def makeModel():
    model = models.Sequential([
        layers.Dense(512, activation='relu', input_shape=(13,)),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        # 출력층: 회귀는 특정 값을 예측하므로 활성화 함수 없이 단일 뉴런으로 구성
        layers.Dense(1)
    ])

    model.compile(optimizer='rmsprop', 
                  loss='mse', 
                  metrics=['mae'])
    return model

network = makeModel()
network.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               7168      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 179713 (702.00 KB)
Trainable params: 179713 (702.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 4. 모델 훈련
`fit()` 메서드를 사용하여 스케일링된 훈련 데이터로 모델을 학습시킵니다.

In [5]:
history = network.fit(X_train_scaled, y_train, epochs=10, batch_size=100, validation_split=0.2, verbose=1)

Epoch 1/10


4/4 [==============================] - 1s 64ms/step - loss: 555.1855 - mae: 21.6911 - val_loss: 549.4913 - val_mae: 21.5606
Epoch 2/10
4/4 [==============================] - 0s 11ms/step - loss: 424.0379 - mae: 18.4214 - val_loss: 275.3605 - val_mae: 13.8652
Epoch 3/10
4/4 [==============================] - 0s 13ms/step - loss: 171.0879 - mae: 9.9360 - val_loss: 99.3993 - val_mae: 6.7789
Epoch 4/10
4/4 [==============================] - 0s 14ms/step - loss: 83.3428 - mae: 6.4144 - val_loss: 82.5983 - val_mae: 6.3260
Epoch 5/10
4/4 [==============================] - 0s 13ms/step - loss: 78.3990 - mae: 6.3643 - val_loss: 84.1561 - val_mae: 6.1527
Epoch 6/10
4/4 [==============================] - 0s 15ms/step - loss: 76.0327 - mae: 6.0889 - val_loss: 79.3529 - val_mae: 5.9441
Epoch 7/10
4/4 [==============================] - 0s 12ms/step - loss: 71.6163 - mae: 5.9167 - val_loss: 88.8493 - val_mae: 6.1457
Epoch 8/10
4/4 [==============================] - 0s 13ms/step - loss: 7

### 5. 모델 평가
`evaluate()` 메서드를 사용하여 훈련된 모델의 성능을 훈련 데이터셋과 테스트 데이터셋에 대해 각각 평가합니다. 손실(MSE)과 평균 절대 오차(MAE)를 출력합니다.

In [6]:
print("\n--- 모델 평가 ---")
train_loss, train_mae = network.evaluate(X_train_scaled, y_train)
test_loss, test_mae = network.evaluate(X_test_scaled, y_test)

print(f"훈련셋   => 손실(MSE): {train_loss:.2f}, MAE: {train_mae:.2f}")
print(f"테스트셋 => 손실(MSE): {test_loss:.2f}, MAE: {test_mae:.2f}")


--- 모델 평가 ---
4/4 [==============================] - 0s 3ms/step - loss: 70.6975 - mae: 5.9713
훈련셋   => 손실(MSE): 69.92, MAE: 5.35
테스트셋 => 손실(MSE): 70.70, MAE: 5.97
